In [1]:
print("cdsc")

cdsc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table
import psycopg2

In [4]:
# подгружаем .env
load_dotenv()

True

In [5]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [6]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [2]:
import boto3
import os

class Config:
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

    S3_SERVICE_NAME = 's3'
    S3_ENDPOINT_URL = 'https://storage.yandexcloud.net'


def get_session():
    session = boto3.session.Session()

    return session.client(
        service_name=Config.S3_SERVICE_NAME,
        endpoint_url=Config.S3_ENDPOINT_URL,
        aws_access_key_id=Config.AWS_ACCESS_KEY_ID,
        aws_secret_access_key=Config.AWS_SECRET_ACCESS_KEY
    )

bucket_name = os.getenv("S3_BUCKET_NAME")

s3 = get_session()

if s3.list_objects(Bucket=bucket_name).get('Contents'):
    for key in s3.list_objects(Bucket=bucket_name)['Contents']:
        print(key['Key'])

TypeError: expected string or bytes-like object

In [3]:
import os
import boto3
from botocore.exceptions import ClientError

# Конфигурация (используйте переменные окружения для секретов!)
S3_ENDPOINT = "https://storage.yandexcloud.net"
S3_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID") # Замените на переменные окружения!
S3_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("S3_BUCKET_NAME")
FILE_KEY = "data/initial_data.csv"
LOCAL_PATH = "./initial_data.csv"  # Куда сохранить файл

# Инициализация клиента
s3 = boto3.client(
    "s3",
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=S3_ACCESS_KEY,
    aws_secret_access_key=S3_SECRET_KEY
)

try:
    # Скачивание файла
    s3.download_file(
        Bucket=BUCKET_NAME,
        Key=FILE_KEY,
        Filename=LOCAL_PATH
    )
    print(f"Файл {FILE_KEY} успешно скачан в {LOCAL_PATH}")

except ClientError as e:
    print(f"Ошибка доступа к S3: {e}")
except Exception as e:
    print(f"Общая ошибка: {e}")

Общая ошибка: expected string or bytes-like object


In [9]:
import boto3
from botocore.exceptions import ClientError

# Конфигурация (используйте переменные окружения для секретов!)
S3_ENDPOINT = "https://storage.yandexcloud.net"
S3_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID") # Замените на переменные окружения!
S3_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY") # Например: os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = "s3-student-mle-20250130-b8bcb9fce1"
FILE_KEY = "data/initial_data.csv"
LOCAL_PATH = "./initial_data.csv"  # Куда сохранить файл

# Инициализация клиента
s3 = boto3.client(
    "s3",
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=S3_ACCESS_KEY,
    aws_secret_access_key=S3_SECRET_KEY
)

try:
    # Скачивание файла
    s3.download_file(
        Bucket=BUCKET_NAME,
        Key=FILE_KEY,
        Filename=LOCAL_PATH
    )
    print(f"Файл {FILE_KEY} успешно скачан в {LOCAL_PATH}")

except ClientError as e:
    print(f"Ошибка доступа к S3: {e}")
except Exception as e:
    print(f"Общая ошибка: {e}")

Файл data/initial_data.csv успешно скачан в ./initial_data.csv


In [10]:
# Загрузка данных
df = pd.read_csv("./initial_data.csv")
X = df.drop("target", axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Обучение модели
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Предсказания
prediction = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]

# Настройка MLflow
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# Создание эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Логирование
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # Логируем метрики
    metrics = {
        "accuracy": accuracy_score(y_test, prediction),
        "precision": precision_score(y_test, prediction),
        "recall": recall_score(y_test, prediction),
        "f1": f1_score(y_test, prediction)
    }
    mlflow.log_metrics(metrics)
    
    # Логируем модель
    signature = mlflow.models.infer_signature(X_test, prediction)
    input_example = X_test[:10]
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example
    )
    
    # Логируем артефакты
    results_df = pd.DataFrame({"true": y_test, "pred": prediction, "proba": proba})
    results_df.to_csv("predictions.csv", index=False)
    mlflow.log_artifact("predictions.csv")

print(f"Run completed: {run.info.run_id}")

# Загрузка модели для проверки
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

SyntaxError: invalid syntax (3407774012.py, line 14)